# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [2]:
import os

pth1 = '/datasets/autos.csv'
pth11 = 'autos.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth11):
    data = pd.read_csv(pth11)
else:
    print('Something is wrong')

pd.set_option('display.max_columns', None)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
max_size = data.shape[0]

Видно, что в некоторых столбцах есть много пропусков

In [5]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Видно косяки в данных. В `RegistrationYear` есть нулевой и 9999-й год. В `Price` есть строки со значением 0. Тоже самое в `Power` и `RegistrationMonth`.

In [6]:
data.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

В `Repaired` самое большое количество пропусков. Предполжим, что это означает, что машина не была в ремонте. В остальном заменим пропуски самыми частыми значениями

In [7]:
data['Repaired'].fillna('no', inplace=True)

In [8]:
brand_list = data['Brand'].unique()
brand_list = list(brand_list)
brand_list

['volkswagen',
 'audi',
 'jeep',
 'skoda',
 'bmw',
 'peugeot',
 'ford',
 'mazda',
 'nissan',
 'renault',
 'mercedes_benz',
 'opel',
 'seat',
 'citroen',
 'honda',
 'fiat',
 'mini',
 'smart',
 'hyundai',
 'sonstige_autos',
 'alfa_romeo',
 'subaru',
 'volvo',
 'mitsubishi',
 'kia',
 'suzuki',
 'lancia',
 'toyota',
 'chevrolet',
 'dacia',
 'daihatsu',
 'trabant',
 'saab',
 'chrysler',
 'jaguar',
 'daewoo',
 'porsche',
 'rover',
 'land_rover',
 'lada']

In [9]:
# for i in brand_list:
#     data.loc[data['Brand'] == i, 'Model'] = data[data['Brand'] == i]['Model'].fillna(data[data['Brand'] == i]['Model'].mode().values[0])

Здесь выходила ошибка. Решил проверить что не так.

In [9]:
k=1
for i in brand_list:
    print(k)
    print(i)
    print(data[data['Brand'] == i]['Model'].mode())
    k=k+1

1
volkswagen
0    golf
Name: Model, dtype: object
2
audi
0    a4
Name: Model, dtype: object
3
jeep
0    grand
Name: Model, dtype: object
4
skoda
0    octavia
Name: Model, dtype: object
5
bmw
0    3er
Name: Model, dtype: object
6
peugeot
0    2_reihe
Name: Model, dtype: object
7
ford
0    focus
Name: Model, dtype: object
8
mazda
0    3_reihe
Name: Model, dtype: object
9
nissan
0    micra
Name: Model, dtype: object
10
renault
0    twingo
Name: Model, dtype: object
11
mercedes_benz
0    c_klasse
Name: Model, dtype: object
12
opel
0    corsa
Name: Model, dtype: object
13
seat
0    ibiza
Name: Model, dtype: object
14
citroen
0    other
Name: Model, dtype: object
15
honda
0    civic
Name: Model, dtype: object
16
fiat
0    punto
Name: Model, dtype: object
17
mini
0    cooper
Name: Model, dtype: object
18
smart
0    fortwo
Name: Model, dtype: object
19
hyundai
0    i_reihe
Name: Model, dtype: object
20
sonstige_autos
Series([], Name: Model, dtype: object)
21
alfa_romeo
0    156
Name: Model, dt

In [10]:
data[data['Brand'] == 'sonstige_autos'].count()

DateCrawled          3374
Price                3374
VehicleType          2333
RegistrationYear     3374
Gearbox              2271
Power                3374
Model                   0
Kilometer            3374
RegistrationMonth    3374
FuelType             2270
Brand                3374
Repaired             3374
DateCreated          3374
NumberOfPictures     3374
PostalCode           3374
LastSeen             3374
dtype: int64

Понял, что проблема в том, что у бренда 'sonstige_autos' нет моделей.

In [11]:
brand_list.remove('sonstige_autos')
brand_list

['volkswagen',
 'audi',
 'jeep',
 'skoda',
 'bmw',
 'peugeot',
 'ford',
 'mazda',
 'nissan',
 'renault',
 'mercedes_benz',
 'opel',
 'seat',
 'citroen',
 'honda',
 'fiat',
 'mini',
 'smart',
 'hyundai',
 'alfa_romeo',
 'subaru',
 'volvo',
 'mitsubishi',
 'kia',
 'suzuki',
 'lancia',
 'toyota',
 'chevrolet',
 'dacia',
 'daihatsu',
 'trabant',
 'saab',
 'chrysler',
 'jaguar',
 'daewoo',
 'porsche',
 'rover',
 'land_rover',
 'lada']

In [12]:
for i in brand_list:
     data.loc[data['Brand'] == i, 'Model'] = data[data['Brand'] == i]['Model'] \
        .fillna(data[data['Brand'] == i]['Model'].mode().values[0])

Чтобы избежать несуществующих сочетаний 'Brand' и 'Model', заменил пропущенные значения в 'Model' модой по моделям конкретного бренда.

In [13]:
data.loc[data['Brand'] == 'sonstige_autos', 'Model'] = data[data['Brand'] == 'sonstige_autos']['Model'].fillna('No')

In [14]:
data.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                    0
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired                 0
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [15]:
model_list = data['Model'].unique()
model_list = list(model_list)
model_list

['golf',
 'a4',
 'grand',
 'fabia',
 '3er',
 '2_reihe',
 'other',
 'c_max',
 '3_reihe',
 'passat',
 'navara',
 'ka',
 'polo',
 'twingo',
 'a_klasse',
 'scirocco',
 '5er',
 'meriva',
 'arosa',
 'c4',
 'civic',
 'transporter',
 'punto',
 'e_klasse',
 'clio',
 'kadett',
 'kangoo',
 'corsa',
 'one',
 'fortwo',
 '1er',
 'b_klasse',
 'signum',
 'astra',
 'No',
 'a8',
 'jetta',
 'fiesta',
 'c_klasse',
 'micra',
 'vito',
 'sprinter',
 '156',
 'escort',
 'forester',
 'xc_reihe',
 'scenic',
 'a1',
 'insignia',
 'combo',
 'focus',
 'tt',
 'a6',
 'jazz',
 'omega',
 'slk',
 '7er',
 '80',
 '147',
 '100',
 'z_reihe',
 'sportage',
 'sorento',
 'v40',
 'ibiza',
 'mustang',
 'eos',
 'touran',
 'getz',
 'a3',
 'almera',
 'megane',
 'lupo',
 'r19',
 'zafira',
 'caddy',
 'mondeo',
 'cordoba',
 'colt',
 'impreza',
 'vectra',
 'berlingo',
 'tiguan',
 'i_reihe',
 'espace',
 'sharan',
 '6_reihe',
 'panda',
 'up',
 'seicento',
 'ceed',
 '5_reihe',
 'yeti',
 'octavia',
 'mii',
 'rx_reihe',
 '6er',
 'modus',
 'fo

In [16]:
for i in model_list:
     data.loc[data['Model'] == i, 'VehicleType'] = data[data['Model'] == i]['VehicleType'] \
        .fillna(data[data['Model'] == i]['VehicleType'].mode().values[0])

Также заменил пропущенные значения в столбце 'VehicleType' модой для каждой модели. Чтобы не вышло несуществующих сочетаний.

In [17]:
data.isnull().sum()

DateCrawled              0
Price                    0
VehicleType              0
RegistrationYear         0
Gearbox              19833
Power                    0
Model                    0
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired                 0
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [18]:
data['FuelType'].unique()

array(['petrol', 'gasoline', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

In [19]:
data = data.replace(['petrol'],'gasoline')
data['FuelType'].unique()

array(['gasoline', nan, 'lpg', 'other', 'hybrid', 'cng', 'electric'],
      dtype=object)

Заменил 'petrol' на 'gasoline', так как это одно и то же.

In [20]:
for i in model_list:
     data.loc[data['Model'] == i, 'FuelType'] = data[data['Model'] == i]['FuelType'] \
        .fillna(data[data['Model'] == i]['FuelType'].mode().values[0])

Также заменил пропущенные значения в столбце 'FuelType' модой для каждой модели. Чтобы не вышло несуществующих сочетаний.

In [21]:
data.isnull().sum()

DateCrawled              0
Price                    0
VehicleType              0
RegistrationYear         0
Gearbox              19833
Power                    0
Model                    0
Kilometer                0
RegistrationMonth        0
FuelType                 0
Brand                    0
Repaired                 0
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [22]:
for i in model_list:
     data.loc[data['Model'] == i, 'Gearbox'] = data[data['Model'] == i]['Gearbox'] \
        .fillna(data[data['Model'] == i]['Gearbox'].mode().values[0])

Также заменил пропущенные значения в столбце 'Gearbox' модой для каждой модели. Чтобы не вышло несуществующих сочетаний.

In [23]:
data.isnull().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

In [24]:
data = data.drop(["DateCrawled","DateCreated","NumberOfPictures","LastSeen", "RegistrationMonth"],axis = 1)

Удалил неинформативные столбцы. Т.к. дата скачивания анкеты из базы, дата создания анкеты, количество фотографий автомобиля, дата последней активности пользователя, месяц регистрации автомобиля логически не связаны со стоимостью автомобиля.

Изначально удалил и 'PostalCode', но получалось слишком много дубликатов. И сам признак действительно может быть важным, так как в разных странах машина может стоить по разному.

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       354369 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           354369 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             354369 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          354369 non-null  object
 8   Brand             354369 non-null  object
 9   Repaired          354369 non-null  object
 10  PostalCode        354369 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


In [26]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,99998.000000


In [27]:
(data[['Price', 'RegistrationYear', 'Power']]
    .quantile([0.0012, 0.9988]))

,Price,RegistrationYear,Power
0.0012,0.0,1960.0,0.0
0.9988,19999.0,2018.0,520.0


In [28]:
data = data[(data['Price'] <= 19999)]
data = data[(data['RegistrationYear'] >= 1960) & (data['RegistrationYear'] <= 2016)]
data = data[(data['Power'] <= 520)]

Откинул выбросы и аномалии

`RegistrationYear` < 2016 т.к. это год выгрузки базы.

In [29]:
data[data['Price'] == 0]['Price'].count()

9915

In [30]:
data[data['Power'] == 0]['Power'].count()

36240

In [31]:
for i in model_list:
     data.loc[((data['Model'] == i) & (data['Price'] == 0)), 'Price'] = data[data['Model'] == i]['Price'].mean()

In [32]:
for i in model_list:
     data.loc[((data['Model'] == i) & (data['Power'] == 0)), 'Power'] = data[data['Model'] == i]['Power'].mean()

Заменил аномальные нулевые значени в столбцах 'Price' и 'Power' на среднее по каждой модели

In [33]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,PostalCode
count,338774.000000,338774.000000,338774.000000,338774.000000,338774.000000
mean,4576.204262,2002.546453,118.169137,128191.891940,50615.570979
std,4471.815507,6.747857,52.484907,37772.412124,25802.436023
min,1.000000,1960.000000,0.000000,5000.000000,1067.000000
25%,1200.000000,1999.000000,75.000000,125000.000000,30173.000000
50%,2984.912050,2003.000000,110.000000,150000.000000,49497.000000
75%,6500.000000,2007.000000,143.000000,150000.000000,71229.000000
max,19999.000000,2016.000000,520.000000,150000.000000,99998.000000


In [34]:
data.duplicated().sum()

27601

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 338774 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             338774 non-null  float64
 1   VehicleType       338774 non-null  object 
 2   RegistrationYear  338774 non-null  int64  
 3   Gearbox           338774 non-null  object 
 4   Power             338774 non-null  float64
 5   Model             338774 non-null  object 
 6   Kilometer         338774 non-null  int64  
 7   FuelType          338774 non-null  object 
 8   Brand             338774 non-null  object 
 9   Repaired          338774 non-null  object 
 10  PostalCode        338774 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 31.0+ MB


In [36]:
data.drop_duplicates(inplace=True)

In [37]:
data.duplicated().sum()

0

Удалили 27601 дубликат

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311173 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             311173 non-null  float64
 1   VehicleType       311173 non-null  object 
 2   RegistrationYear  311173 non-null  int64  
 3   Gearbox           311173 non-null  object 
 4   Power             311173 non-null  float64
 5   Model             311173 non-null  object 
 6   Kilometer         311173 non-null  int64  
 7   FuelType          311173 non-null  object 
 8   Brand             311173 non-null  object 
 9   Repaired          311173 non-null  object 
 10  PostalCode        311173 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 28.5+ MB


In [39]:
samp_decrease = data.shape[0]/max_size
samp_decrease

0.8781044617333909

Удалено практически 12.2% данных

In [40]:
data.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired,PostalCode
0,480.0,sedan,1993,manual,93.209522,golf,150000,gasoline,volkswagen,no,70435
1,18300.0,coupe,2011,manual,190.000000,a4,125000,gasoline,audi,yes,66954
2,9800.0,suv,2004,auto,163.000000,grand,125000,gasoline,jeep,no,90480
3,1500.0,small,2001,manual,75.000000,golf,150000,gasoline,volkswagen,no,91074
4,3600.0,small,2008,manual,69.000000,fabia,90000,gasoline,skoda,no,60437


In [42]:
RANDOM_STATE=42
TEST_SIZE=0.2
VALID_TEST=0.25

X = data.drop(['Price'], axis=1)
y = data['Price']

X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=VALID_TEST, random_state=RANDOM_STATE)

X_train_forest = X_train.copy()
X_test_forest = X_test.copy()
 
cat_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

ordinal_encoder = OrdinalEncoder()

X_train_forest[cat_columns] = ordinal_encoder.fit_transform(X_train_forest[cat_columns])

X_test_forest[cat_columns] = ordinal_encoder.transform(X_test_forest[cat_columns])


Разбил данные на train, test и valid. Также сделал копии train и test для случайного леса и закодировал категориальные переменные в них с помощью OrdinalEncoder.

OrdinalEncoder использую, так как модель случайного леса основана на решающих деревьях, для которых, в отличии от линейных моделей, можно использовать данный кодировщик

## Обучение моделей

### 1. Обучение и оценка модели 

#### Случайный лес

In [43]:
%%time

forest_model = RandomForestRegressor(random_state=RANDOM_STATE)

parameters = {'n_estimators': range(10,51,10),
              'max_depth': range(4,8)}

gs = GridSearchCV(forest_model,
                  parameters,
                  n_jobs=-1,
                  cv=5,
                  scoring='neg_root_mean_squared_error')

gs.fit(X_train_forest, y_train)

print('Гиперпараметры', gs.best_params_)
print('RMSE', gs.best_score_)

Гиперпараметры {'max_depth': 7, 'n_estimators': 30}
RMSE -2170.7630113588843
Wall time: 2min 22s


In [44]:
%%time
final_forest = RandomForestRegressor(n_estimators=gs.best_params_['n_estimators'],
                                     max_depth=gs.best_params_['max_depth'],
                                     random_state=RANDOM_STATE)
final_forest.fit(X_train_forest, y_train)

Wall time: 5.48 s


RandomForestRegressor(max_depth=7, n_estimators=30, random_state=42)

С помощью GridSearchCV посчитал лучшие значения гиперпараметров и обучил по ним модель случайного леса на тренировочных данных

#### CatBoost

Для CatBoostRegressor не кодирую данные с помощью стандартных кодировщиков (OHE или OrdinalEncoder), так как есть встроенный - cat_features, в котором нужно просто указать список категориальных столбцов.

In [45]:
%%time
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

catboost = CatBoostRegressor(cat_features=cat_features, loss_function='RMSE', random_seed=RANDOM_STATE)

grid = {'n_estimators': [100, 1000, 1100],
        'max_depth': [4, 6],
        'learning_rate': [0.0001, 0.03, 0.1]}

grid_search_result = catboost.grid_search(grid,
                                          X=X_train,
                                          y=y_train,
                                          cv=3,
                                          plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 6381.3377455	test: 6401.4030816	best: 6401.4030816 (0)	total: 195ms	remaining: 19.3s
1:	learn: 6380.8328539	test: 6400.8956447	best: 6400.8956447 (1)	total: 231ms	remaining: 11.3s
2:	learn: 6380.3266282	test: 6400.3863697	best: 6400.3863697 (2)	total: 267ms	remaining: 8.63s
3:	learn: 6379.8170154	test: 6399.8743268	best: 6399.8743268 (3)	total: 301ms	remaining: 7.22s
4:	learn: 6379.3110814	test: 6399.3644988	best: 6399.3644988 (4)	total: 337ms	remaining: 6.4s
5:	learn: 6378.8057354	test: 6398.8565119	best: 6398.8565119 (5)	total: 379ms	remaining: 5.94s
6:	learn: 6378.3048752	test: 6398.3516098	best: 6398.3516098 (6)	total: 416ms	remaining: 5.53s
7:	learn: 6377.8010169	test: 6397.8447817	best: 6397.8447817 (7)	total: 452ms	remaining: 5.2s
8:	learn: 6377.2955879	test: 6397.3374275	best: 6397.3374275 (8)	total: 482ms	remaining: 4.87s
9:	learn: 6376.7928955	test: 6396.8313119	best: 6396.8313119 (9)	total: 512ms	remaining: 4.61s
10:	learn: 6376.2908532	test: 6396.3254160	best: 639

86:	learn: 6338.1530259	test: 6357.9699101	best: 6357.9699101 (86)	total: 2.87s	remaining: 429ms
87:	learn: 6337.6523003	test: 6357.4664822	best: 6357.4664822 (87)	total: 2.9s	remaining: 396ms
88:	learn: 6337.1531781	test: 6356.9644434	best: 6356.9644434 (88)	total: 2.93s	remaining: 362ms
89:	learn: 6336.6567316	test: 6356.4636573	best: 6356.4636573 (89)	total: 2.96s	remaining: 329ms
90:	learn: 6336.1589727	test: 6355.9619005	best: 6355.9619005 (90)	total: 2.99s	remaining: 296ms
91:	learn: 6335.6605462	test: 6355.4612053	best: 6355.4612053 (91)	total: 3.02s	remaining: 263ms
92:	learn: 6335.1674211	test: 6354.9656510	best: 6354.9656510 (92)	total: 3.05s	remaining: 230ms
93:	learn: 6334.6696347	test: 6354.4638093	best: 6354.4638093 (93)	total: 3.08s	remaining: 197ms
94:	learn: 6334.1758812	test: 6353.9679624	best: 6353.9679624 (94)	total: 3.11s	remaining: 164ms
95:	learn: 6333.6827775	test: 6353.4729677	best: 6353.4729677 (95)	total: 3.14s	remaining: 131ms
96:	learn: 6333.1830548	test: 6

75:	learn: 2414.7675010	test: 2413.6944922	best: 2413.6944922 (75)	total: 2.47s	remaining: 779ms
76:	learn: 2404.5346753	test: 2403.5308749	best: 2403.5308749 (76)	total: 2.51s	remaining: 751ms
77:	learn: 2395.3253900	test: 2394.4117375	best: 2394.4117375 (77)	total: 2.55s	remaining: 719ms
78:	learn: 2386.7835548	test: 2385.7779031	best: 2385.7779031 (78)	total: 2.58s	remaining: 687ms
79:	learn: 2378.0529112	test: 2376.9753735	best: 2376.9753735 (79)	total: 2.62s	remaining: 655ms
80:	learn: 2369.6955406	test: 2368.7262052	best: 2368.7262052 (80)	total: 2.65s	remaining: 622ms
81:	learn: 2361.7702814	test: 2361.0532176	best: 2361.0532176 (81)	total: 2.69s	remaining: 589ms
82:	learn: 2353.4362785	test: 2352.8198558	best: 2352.8198558 (82)	total: 2.73s	remaining: 558ms
83:	learn: 2345.2391278	test: 2344.8572947	best: 2344.8572947 (83)	total: 2.76s	remaining: 526ms
84:	learn: 2337.8204699	test: 2337.2262767	best: 2337.2262767 (84)	total: 2.79s	remaining: 493ms
85:	learn: 2330.9742118	test: 

66:	learn: 2007.9787724	test: 2022.9135432	best: 2022.9135432 (66)	total: 2.04s	remaining: 1s
67:	learn: 2004.4176915	test: 2019.2550611	best: 2019.2550611 (67)	total: 2.07s	remaining: 975ms
68:	learn: 2001.3723204	test: 2016.0633430	best: 2016.0633430 (68)	total: 2.1s	remaining: 944ms
69:	learn: 1999.0449248	test: 2013.9863160	best: 2013.9863160 (69)	total: 2.13s	remaining: 914ms
70:	learn: 1997.4007797	test: 2012.6099903	best: 2012.6099903 (70)	total: 2.16s	remaining: 883ms
71:	learn: 1994.9773759	test: 2009.9757896	best: 2009.9757896 (71)	total: 2.19s	remaining: 852ms
72:	learn: 1992.1287456	test: 2006.9164342	best: 2006.9164342 (72)	total: 2.22s	remaining: 821ms
73:	learn: 1990.4902153	test: 2005.2490697	best: 2005.2490697 (73)	total: 2.25s	remaining: 791ms
74:	learn: 1988.7290217	test: 2003.9450824	best: 2003.9450824 (74)	total: 2.28s	remaining: 761ms
75:	learn: 1986.7783691	test: 2002.2442960	best: 2002.2442960 (75)	total: 2.31s	remaining: 730ms
76:	learn: 1984.9210942	test: 2000

...

702:	learn: 1651.5312091	test: 1750.3028726	best: 1750.3028726 (702)	total: 48.6s	remaining: 27.5s
703:	learn: 1651.3866644	test: 1750.2366061	best: 1750.2366061 (703)	total: 48.7s	remaining: 27.4s
704:	learn: 1651.1728761	test: 1750.1514877	best: 1750.1514877 (704)	total: 48.8s	remaining: 27.3s
705:	learn: 1650.9969297	test: 1750.0612537	best: 1750.0612537 (705)	total: 48.8s	remaining: 27.2s
706:	learn: 1650.7879949	test: 1749.9879610	best: 1749.9879610 (706)	total: 48.9s	remaining: 27.2s
707:	learn: 1650.4822058	test: 1749.7908014	best: 1749.7908014 (707)	total: 48.9s	remaining: 27.1s
708:	learn: 1650.1599133	test: 1749.6378619	best: 1749.6378619 (708)	total: 49s	remaining: 27s
709:	learn: 1649.9884412	test: 1749.5773210	best: 1749.5773210 (709)	total: 49.1s	remaining: 27s
710:	learn: 1649.8612517	test: 1749.3891125	best: 1749.3891125 (710)	total: 49.2s	remaining: 26.9s
711:	learn: 1649.6901612	test: 1749.2683980	best: 1749.2683980 (711)	total: 49.2s	remaining: 26.8s
712:	learn: 1649

786:	learn: 1639.7479174	test: 1745.5518714	best: 1745.5518714 (786)	total: 54.7s	remaining: 21.7s
787:	learn: 1639.5812177	test: 1745.4963338	best: 1745.4963338 (787)	total: 54.8s	remaining: 21.7s
788:	learn: 1639.3780366	test: 1745.3942889	best: 1745.3942889 (788)	total: 54.8s	remaining: 21.6s
789:	learn: 1639.3487538	test: 1745.3903978	best: 1745.3903978 (789)	total: 54.9s	remaining: 21.5s
790:	learn: 1639.1280185	test: 1745.2186929	best: 1745.2186929 (790)	total: 55s	remaining: 21.5s
791:	learn: 1639.0140433	test: 1745.1947929	best: 1745.1947929 (791)	total: 55s	remaining: 21.4s
792:	learn: 1638.9162694	test: 1745.1632996	best: 1745.1632996 (792)	total: 55.1s	remaining: 21.3s
793:	learn: 1638.7438562	test: 1745.0228376	best: 1745.0228376 (793)	total: 55.2s	remaining: 21.3s
794:	learn: 1638.5590002	test: 1745.0329731	best: 1745.0228376 (793)	total: 55.2s	remaining: 21.2s
795:	learn: 1638.3032902	test: 1744.9996004	best: 1744.9996004 (795)	total: 55.3s	remaining: 21.1s
796:	learn: 16

871:	learn: 1627.6938140	test: 1740.5921091	best: 1740.5921091 (871)	total: 1m	remaining: 15.9s
872:	learn: 1627.6169507	test: 1740.5692719	best: 1740.5692719 (872)	total: 1m	remaining: 15.8s
873:	learn: 1627.5727296	test: 1740.5729080	best: 1740.5692719 (872)	total: 1m	remaining: 15.7s
874:	learn: 1627.4308771	test: 1740.5364614	best: 1740.5364614 (874)	total: 1m	remaining: 15.7s
875:	learn: 1627.3258559	test: 1740.5546438	best: 1740.5364614 (874)	total: 1m 1s	remaining: 15.6s
876:	learn: 1627.1980252	test: 1740.5144023	best: 1740.5144023 (876)	total: 1m 1s	remaining: 15.5s
877:	learn: 1627.0358779	test: 1740.4860295	best: 1740.4860295 (877)	total: 1m 1s	remaining: 15.5s
878:	learn: 1626.8956845	test: 1740.3512210	best: 1740.3512210 (878)	total: 1m 1s	remaining: 15.4s
879:	learn: 1626.7980826	test: 1740.3575418	best: 1740.3512210 (878)	total: 1m 1s	remaining: 15.3s
880:	learn: 1626.7141945	test: 1740.4245214	best: 1740.3512210 (878)	total: 1m 1s	remaining: 15.3s
881:	learn: 1626.60110

956:	learn: 1618.6433685	test: 1737.0873173	best: 1737.0717637 (955)	total: 1m 6s	remaining: 9.93s
957:	learn: 1618.4772960	test: 1736.9995933	best: 1736.9995933 (957)	total: 1m 6s	remaining: 9.86s
958:	learn: 1618.4641387	test: 1736.9689893	best: 1736.9689893 (958)	total: 1m 6s	remaining: 9.79s
959:	learn: 1618.2727465	test: 1736.9652664	best: 1736.9652664 (959)	total: 1m 6s	remaining: 9.72s
960:	learn: 1618.1628462	test: 1736.9123994	best: 1736.9123994 (960)	total: 1m 6s	remaining: 9.65s
961:	learn: 1618.1045794	test: 1736.8606845	best: 1736.8606845 (961)	total: 1m 6s	remaining: 9.58s
962:	learn: 1617.9678746	test: 1736.8179670	best: 1736.8179670 (962)	total: 1m 6s	remaining: 9.52s
963:	learn: 1617.8854375	test: 1736.9054702	best: 1736.8179670 (962)	total: 1m 6s	remaining: 9.45s
964:	learn: 1617.7280419	test: 1736.7762913	best: 1736.7762913 (964)	total: 1m 7s	remaining: 9.38s
965:	learn: 1617.6739004	test: 1736.7836490	best: 1736.7762913 (964)	total: 1m 7s	remaining: 9.31s
966:	learn

1038:	learn: 1609.4593363	test: 1733.6778113	best: 1733.6778113 (1038)	total: 1m 12s	remaining: 4.24s
1039:	learn: 1609.3734161	test: 1733.7355120	best: 1733.6778113 (1038)	total: 1m 12s	remaining: 4.17s
1040:	learn: 1609.3013525	test: 1733.7785281	best: 1733.6778113 (1038)	total: 1m 12s	remaining: 4.1s
1041:	learn: 1609.1651933	test: 1733.6499691	best: 1733.6499691 (1041)	total: 1m 12s	remaining: 4.04s
1042:	learn: 1608.9337206	test: 1733.4227974	best: 1733.4227974 (1042)	total: 1m 12s	remaining: 3.96s
1043:	learn: 1608.8769045	test: 1733.4264317	best: 1733.4227974 (1042)	total: 1m 12s	remaining: 3.9s
1044:	learn: 1608.8390223	test: 1733.4219981	best: 1733.4219981 (1044)	total: 1m 12s	remaining: 3.83s
1045:	learn: 1608.7104759	test: 1733.3223985	best: 1733.3223985 (1045)	total: 1m 12s	remaining: 3.76s
1046:	learn: 1608.5730885	test: 1733.2102200	best: 1733.2102200 (1046)	total: 1m 12s	remaining: 3.69s
1047:	learn: 1608.4699285	test: 1733.1575432	best: 1733.1575432 (1047)	total: 1m 12s

In [46]:
grid_search_result["params"]

{'depth': 6, 'iterations': 1100, 'learning_rate': 0.1}

In [47]:
%%time
final_cat = CatBoostRegressor(cat_features=cat_features,
                              n_estimators=grid_search_result["params"]["iterations"],
                              learning_rate=grid_search_result["params"]['learning_rate'],
                              max_depth=grid_search_result["params"]['depth'],
                              loss_function='RMSE',
                              verbose=100,
                              random_seed=RANDOM_STATE)
final_cat.fit(X_train, y_train)

0:	learn: 4202.8079486	total: 84.8ms	remaining: 1m 33s
100:	learn: 1844.5187974	total: 7.89s	remaining: 1m 18s
200:	learn: 1775.5053368	total: 15.5s	remaining: 1m 9s
300:	learn: 1739.6536266	total: 23.2s	remaining: 1m 1s
400:	learn: 1716.6408181	total: 31s	remaining: 54s
500:	learn: 1697.0134567	total: 39.2s	remaining: 46.9s
600:	learn: 1680.7408504	total: 47s	remaining: 39.1s
700:	learn: 1667.2558108	total: 55s	remaining: 31.3s
800:	learn: 1654.5367972	total: 1m 2s	remaining: 23.5s
900:	learn: 1643.2429951	total: 1m 11s	remaining: 15.7s
1000:	learn: 1633.6749441	total: 1m 19s	remaining: 7.84s
1099:	learn: 1624.6394569	total: 1m 27s	remaining: 0us
Wall time: 1min 28s


С помощью grid_search подобрал лучшие гиперпараметры для CatBoostRegressor и обучил по ним модель на тренировочных данных. Процесс подбора гиперпараметров долгий - 18 минут

#### LightGBM

In [48]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

In [49]:
for cat in cat_features:
    X_train[cat] = X_train[cat].astype('category')
    X_valid[cat] = X_valid[cat].astype('category')    

In [50]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186703 entries, 307413 to 127259
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   VehicleType       186703 non-null  category
 1   RegistrationYear  186703 non-null  int64   
 2   Gearbox           186703 non-null  category
 3   Power             186703 non-null  float64 
 4   Model             186703 non-null  category
 5   Kilometer         186703 non-null  int64   
 6   FuelType          186703 non-null  category
 7   Brand             186703 non-null  category
 8   Repaired          186703 non-null  category
 9   PostalCode        186703 non-null  int64   
dtypes: category(6), float64(1), int64(3)
memory usage: 8.4 MB


In [51]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62235 entries, 168830 to 304847
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   VehicleType       62235 non-null  category
 1   RegistrationYear  62235 non-null  int64   
 2   Gearbox           62235 non-null  category
 3   Power             62235 non-null  float64 
 4   Model             62235 non-null  category
 5   Kilometer         62235 non-null  int64   
 6   FuelType          62235 non-null  category
 7   Brand             62235 non-null  category
 8   Repaired          62235 non-null  category
 9   PostalCode        62235 non-null  int64   
dtypes: category(6), float64(1), int64(3)
memory usage: 2.8 MB


In [52]:
from lightgbm import LGBMRegressor

In [53]:
%%time
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

parameters = {'max_depth': [4, 6],
              'learning_rate': [0.0001, 0.03, 0.1],
              'n_estimators': [100, 1000, 1100]}

gbm = LGBMRegressor(max_depth=3,
                    learning_rate=0.1,
                    n_estimators=200,
                    seed=RANDOM_STATE)

gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='neg_root_mean_squared_error', cv=3)
gsearch.fit(X_train, y_train)

print("Best score: %0.3f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003702 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 875
[LightGBM] [Info] Number of data points in the train set: 124468, number of used features: 10
[LightGBM] [Info] Start training from score 4549.328779
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002775 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 879
[LightGBM] [Info] Number of data points in the train set: 124469, number of used features: 10
[LightGBM] [Info] Start training from score 4536.763433
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002892 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002308 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 875
[LightGBM] [Info] Number of data points in the train set: 124468, number of used features: 10
[LightGBM] [Info] Start training from score 4549.328779
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly se

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Accu

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

С помощью GridSearchCV подобрал лучшие гиперпараметры для LGBMRegressor и обучил по ним модель на тренировочных данных. Процесс подбора гиперпараметров значительно быстрее, чем у CatBoost - 2 минут

In [54]:
%%time
final_gbm = LGBMRegressor(n_estimators=best_parameters['n_estimators'],
                          learning_rate=best_parameters['learning_rate'],
                          max_depth=best_parameters['max_depth'])
final_gbm.fit(X_train, y_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004651 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 883
[LightGBM] [Info] Number of data points in the train set: 186703, number of used features: 10
[LightGBM] [Info] Start training from score 4545.694974
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

LGBMRegressor(max_depth=6, n_estimators=1100)

Обучил модель LGBMRegressor на тренировочных данных

In [55]:
%%time
print("RandomForest:", root_mean_squared_error(y_train, final_forest.predict(X_train_forest)))

RandomForest: 2153.739151728246
Wall time: 314 ms


In [56]:
%%time
print("CatBoost:", root_mean_squared_error(y_train, final_cat.predict(X_train)))

CatBoost: 1628.9431358352704
Wall time: 332 ms


In [57]:
for cat in cat_features:
    X_test[cat] = X_test[cat].astype('category')

In [58]:
%%time
print("LightGBM:", root_mean_squared_error(y_train, final_gbm.predict(X_train)))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
LightGBM: 1441.3494784183679
Wall time: 3.96 s


На train добились лучшего значения RMSE с помощью LightGBM

Теперь проведем тестирование лучшей модели для контроля переобучения

In [59]:
%%time
lightgbm_test_predictions = final_gbm.predict(X_test)
lightgbm_test_rmse = root_mean_squared_error(y_test, lightgbm_test_predictions)
print("LightGBM RMSE на тестовой выборке:", lightgbm_test_rmse)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
LightGBM RMSE на тестовой выборке: 1690.3743811771333
Wall time: 1.35 s


## Анализ моделей

Необходимо было с помощью датасета с информацией сервиса по продаже автомобилей предсказать цену автомобиля.       
Мной были обработаны аномалии, выбросы и дубликаты. Неинформативные столбцы удалены.      
Данные разбиты на части и закодированы.    
Мы обучили три модели: `LGBMRegressor`, `CatBoostRegressor` и `RandomForestRegressor`. По качеству предсказания выиграла модель LightGBM. По времени обучения, считая перебор гиперпараметров, - случайный лес. А по времени предсказания - CatBoost.
Учитывая очень долго обучение CatBoost и плохое значение RMSE у RandomForestRegressor, в качестве лучшей модели выбрал `LGBMRegressor` - RMSE на тестовой выборке: 1690.3743811771333